# Iris Classification
This notebook gets you hands-on with Seldon Deploy, demonstrating how you can deploy, monitor, manage and explain machine learning models. 

In this notebook you will:

* Explore the Iris dataset
* Train several models on the dataset
* Deploy trained models to Seldon
* Train an anchor tabular explainer and update Seldon deployment with this explainer
* Train an outlier detector (variational autoencoder) and update deployment


For each of the machine learning components (the models themselves, explainer and outlier detector) the workflow is the same: 
1. Train algorithm
2. Push trained artefact to Google Storage bucket
3. Create deployment on Seldon Deploy via API

### Import relevant packages

In [ ]:
!pip install seldon-deploy-sdk
!pip install alibi
!pip install alibi-detect
!pip install fsspec
!pip install gcsfs
!pip install dill

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

import numpy as np
import pandas as pd

from alibi.explainers import AnchorTabular

from alibi_detect.od import Mahalanobis
from alibi_detect.datasets import fetch_kdd
from alibi_detect.utils.data import create_outlier_batch
from alibi_detect.utils.mapping import ord2ohe
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.visualize import plot_instance_score, plot_roc

import dill
import os
import joblib

import matplotlib.pyplot as plt
import xgboost as xgb
# import seaborn as sns

### Loading the data
Once you have all of the relevant packages you can now explore the dataset. 

In [ ]:
dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

X = dataset.data
y = dataset.target

Creating a train/test split of the data to ensure you have an unseen subset of the dataset with which to validate your model's performance. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train[0]

Inspecting a single instance (row) of your data, you can see that it is made up of four different numerical features:
1. Sepal length
2. Sepal width
3. Petal length
4. Petal width

In [ ]:
np.unique(y_train)

Meanwhile, the labels (your y data) have three separate classes- each representing a different species of Iris:

0. Iris setosa
1. Iris virginica
2. Iris versicolor

I wouldn't worry about remembering those names...

### Model training
Next you will train two separate machine learning models on your dataset. This is representative of the iterative process of algorithm creation which Data Scientists go through during the experimentation process. 

The first model is a logistic regressor which we train using the `.fit()` method, and then evaluate using the `accuracy_score()` function. 

In [ ]:
lr = LogisticRegression(max_iter=4000)
lr.fit(X_train, y_train)

print(precision_score(y_test, lr.predict(X_test), average="macro"))
print(recall_score(y_test, lr.predict(X_test), average="macro"))
print(accuracy_score(y_test, lr.predict(X_test)))

You'll notice that your model has perfect evaluation scores, this implies that the model is overfitted on the dataset. For the purposes of this workshop, you're not going to worry about this.

----

You can now train your second machine learning model- an XGBoost classifier. The first step is to convert the dataset into XGBoost's expected DMatrix data format. 

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

Next, you will set the hyperparameters associated with the XGBoost algorithim. This is essentially telling the model how large to be (`max_depth`), how quickly to learn (`eta`), information about the task (`objective`) and  the number of classes (`num_class`). 

Finally, you set the `num_round` which tells XGBoost how many iterations over the dataset to perform. 

In [ ]:
param = {
    'max_depth': 3,
    'eta': 0.3,
    'objective': 'multi:softprob',
    'num_class': 3} 

num_round = 20

You can then train and score the classifier. 

In [ ]:
bst = xgb.train(param, dtrain, num_round)
preds = np.asarray([np.argmax(line) for line in bst.predict(dtest)])
print(precision_score(y_test, preds, average="macro"))
print(recall_score(y_test, preds, average="macro"))
print(accuracy_score(y_test, preds))

Again, you can see that the model has overfit on the dataset. However, you will now save both of the models ready for them to be deployed. 

The Scikit-Learn pre-packaged server expects the saved model artefact to be called `model.joblib`. Meanwhile, the XGBoost pre-packaged server expects your model to be called `model.bst` when it is uploaded to our storage bucket.

In [ ]:
# Saving the logistic regressor
joblib.dump(lr, 'model.joblib')

# Saving the XGBoost classifier
bst.save_model('model.bst')

### Push model artefacts to GCP

You will now push the saved model binaries to a GCP bucket, where they can be picked up by Seldon and deployed onto Kubernetes. 

You will need to create a unique name for your model artefact. It's easiest to just use your own name. Be careful not to use any upper-case letters or other characters like "_". Dashes are fine, so for example: YOUR_NAME = "john-smith"

In [ ]:
YOUR_NAME = "john-smith"

In [ ]:
!gsutil cp model.joblib gs://tom-seldon-examples/deploy-workshop/"{YOUR_NAME}"/lr/model.joblib
!gsutil cp model.bst gs://tom-seldon-examples/deploy-workshop/"{YOUR_NAME}"/xgb/model.bst

### Model Deployment

Deploying the model to a Seldon Deploy trial instance using the `seldon-deploy-sdk`. 

First, setting up the configuration and authentication required to access the cluster. 

In [ ]:
SD_IP = ""

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate("admin@seldon.io", "12341234")
    api_client = ApiClient(config)
    return api_client

Now we have configured the IP correctly as well as setup our authentication function we can describe the deployment we would like to create.

You will need to fill in the DEPLOYMENT_NAME, NAMESPACE, and the MODEL_LOCATION, the rest of the deployment description has been templated for you.

We fill first deploy the logistic regression model:

In [ ]:
MODEL_NAME = "lr"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/{MODEL_NAME}"

In [ ]:
NAMESPACE = "test"
PREPACKAGED_SERVER = "SKLEARN_SERVER"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Our model should now be running as a fully fledged microservice. You can now log into Seldon Deploy and test your deployment:

* URL: http://SD_IP/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234

You can now test your model with this request.
```
{
    "data": {
    "names": ["Sepal length","Sepal width","Petal length", "Petal Width"],
    "ndarray": [
        [6.8,  2.8,  4.8,  1.4]
    ]
    }
}
```

Next, you will add your XGBoost model as a canary and promote that to be the main predictor through the UI. Make sure to select 'XGBoost' as the runtime and use the following model artefact address:  

In [ ]:
print(f"gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/{MODEL_NAME}")

# Adding a Model Explainer

Next, we shall train an explainer to glean deeper insights into the decisions being made by our model. We will make use of the [Anchors algorithm](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html), which has a production grade implementation available using the Seldon Alibi Explain library.

The algorithm provides model-agnostic (black box) and human interpretable explanations suitable for classification models applied to images, text and tabular data. The idea behind anchors is to explain the behaviour of complex models with high-precision rules called anchors. These anchors are locally sufficient conditions to ensure a certain prediction with a high degree of confidence. Anchor algorithms incorporate coverage, which is the region the explanation applies within and are optimised to maximize coverage.

As an example of anchors for tabular data, if we want to predict whether a person makes less or more than £50,000 per year based on the person’s characteristics including age (continuous variable) and marital status (categorical variable), then the following would be a potential anchor: Hugo makes more than £50,000 because he is married and his age is between 35 and 45 years.

The first step will be to write a simple prediction function which the explainer can call in order to query our logistic regression model.

In [ ]:
predict_fn = lambda x: lr.predict(x)

Now, define and fit the AnchorTabular explainer. 

In [ ]:
explainer = AnchorTabular(predict_fn, feature_names)
explainer.fit(X_train, disc_perc=(25, 50, 75))

Predicting the first instance within the test set, and consequently generating an explanation for that prediction.

In [ ]:
idx = 0
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

In [ ]:
X_test[0]

Save the explainer. Your explainer must be saved as `explainer.dill` as once again Seldon Deploy will look for this artefact within a top level directory. 

NOTE: Dill is used to serialise the object instead of pickle as it offers a greater flexibilty in the object types which can be serialised. 

In [ ]:
dill.dump(explainer, open( "explainer.dill", "wb"))

Uploading the explainer to a Google Storage bucket. 

In [ ]:
!gsutil cp explainer.dill gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/lr/explainer.dill

We will not update our deployment with the model trained in this notebook as Alibi Explain requires explainers to be trained within a Python 3.6.1 environment which Google Colab currently does not offer. We'll use a pretrained model instead.

In [ ]:
EXPLAINER_TYPE = "AnchorTabular"
EXPLAINER_URI = f"gs://tom-seldon-examples/deploy-workshop/pretrained"

explainer_spec = {
                    "type": EXPLAINER_TYPE,
                    "modelUri": EXPLAINER_URI,
                    "containerSpec": {
                        "name": "",
                        "resources": {}
                    }
                }

In [ ]:
mldeployment['spec']['predictors'][0]['explainer'] = explainer_spec
mldeployment

Finally, creating our new deployment with the explainer added. 

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

# Outlier Detection

In this example you will make use of a Mahalanbois outlier detector. This algorithm calculates an outlier score, which is a measure of distance from the center of the features distribution (Mahalanobis distance). The Mahalanobis distance is focused on the idea of measuring how many standard deviations away your data point is from the mean of the distribution. 

The Mahalanobis outlier detector is an online detector, meaning that it has a running mean and covariance matrix with which it compares the new data points with to classify as outliers or not. It is online in the sense that each new data point contributes to this running mean and covariance matrix.

The first step is to initialise your outlier detector. There are 4 parameters which will be critical to the performance of your outlier detector: 
* `threshold`: The distance threshold above which your data instance is flagged as an outlier. To begin with you can leave this blank and infer a given threshold later when fitting your detector to the dataset. 
* `n_components`: The number of components to use within the PCA. PCA is used to reduce the number of feature dimensions. This will make it easier to compute the Mahalanbois calculation especially on high feature datasets. 
* `std_clip`: Feature-wise standard deviation used to clip the observations before updating the mean and covariance matrix. 
* `start_clip`: Number of observations before clipping is applied.

If your data has categorical features then these can be specified using the `cat_vars` parameter. 

Firstly, you will set the `n_components` value. The simplest way to set this value is using `n_components == min(n_samples, n_features)` and then scaling down if computational performance slows the computation of outliers.

A more rigourous approach is demonstrated below. This essentially calculates the covariance matrix- how each feature varies with every other feature. This determines the eigenvalues and eigenvectors of the covariance matrix. 

The eigenvalues capture the variance of each component in the direction of the eigenvector. Therefore, you can use this to understand the contribution of each feature to the overall variance. From here you can make a determination of how many components to include in your PCA, as you now understand the contribution of each of the features. 

In [ ]:
mean, stdev = X_train.mean(axis=0), X_train.std(axis=0)
cov_mat = np.cov((X_train - mean).T)
cov_mat

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
print(f'Eigenvectors \n {eig_vecs}')
print(f'Eigenvectors \n {eig_vals}')

In [ ]:
total = sum(eig_vals)
var_exp = [(i / total)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

print("Variance captured by each component:")
print(var_exp)
print(80 * "-")
print("Cumulative variance captured as we travel each component:")
print(cum_var_exp)

From the above you can see that 91.9% of the variance is captured by a single component, therefore you could reasonably set `n_components = 1`. However, you gain an additional 5% points by including a second one, and therefore:

In [ ]:
n_components = 2

`std_clip` is used to ensure that outliers do not impact the typical operation of the detector. If many, many outliers (e.g. in a DDoS attack, or multiple broken parts in a assembly line) were sent to the detector then the weights (covariance matrix) would shift to include these outliers, therefore we set a `std_clip` whereby any data points above this clip level will have their `stdev` clipped to this maximal value. 

Setting the `std_clip` is case by case dependent, but you will use the following heuristic to set it. You will look at the `stdev` across the features, selecting the one which is the most widely distributed (the highest value). You will then set the `std_clip` to be 2 stdevs away from this maximal value. 

The code snippet below illustrates this.

In [ ]:
std_clip = (2 * stdev.max(axis=0)).round(decimals=2)
std_clip

Finally, the `start_clip` value is used to "warm up" the outlier detector. When the detector is first initialised it can be sensitive to new observations- which lead to large changes in the online mean and covariance matrix. During this period of time you do not want to show the outlier detector any outliers as it can greatly skew the accuracy of the running mean and covariance matrix.

Thus during this "warm up" period you only show the outlier detector examples of normal data, and do not perform any clipping. The `start_clip` value determines after which point you should begin clipping data points and introducing outliers into the mix. 

Therefore `start_clip` in this case will be equal to the length of `X_train`, as you know that there are no outlying data points within the original Iris dataset. 

In [ ]:
start_clip = len(X_train)
start_clip

In [ ]:
threshold = None

od = Mahalanobis(threshold,
                 n_components=n_components,
                 std_clip=std_clip,
                 start_clip=start_clip)

If you were dealing with categorical variables you would now run `od.fit(...)` however because the Iris dataset is solely continuous variables you do not need to perform this step. 

------

You do however now need to set the threshold of Mahalanobis distance above which you consider a data instance an outlier. This can be done using the convenient `infer_threshold()` method. 

You first create a synthetic outlier batch using your training data and labels, with a certain percentage of the batch as outliers. Based upon this percentage of outliers you shall update the threshold value.

In [ ]:
np.random.seed(0)
perc_outlier = 5

threshold_batch = create_outlier_batch(X_train, y_train, n_samples=1000, perc_outlier=perc_outlier)
X_threshold, y_threshold = threshold_batch.data.astype('float'), threshold_batch.target
X_threshold = (X_threshold - mean) / stdev
print('{}% outliers'.format(100 * y_threshold.mean()))

Once you have created the synthetic data we can determine the threshold using `infer_threshold`:

In [ ]:
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))
threshold = od.threshold

You can now test your threshold by generating a second batch of outlying data, this time with a higher proportion of outliers: 

In [ ]:
np.random.seed(1)
outlier_batch = create_outlier_batch(X_train, y_train, n_samples=1000, perc_outlier=10)
X_outlier, y_outlier = outlier_batch.data.astype('float'), outlier_batch.target
X_outlier = (X_outlier - mean) / stdev
print(X_outlier.shape, y_outlier.shape)
print('{}% outliers'.format(100 * y_outlier.mean()))

Generating outlier predictions from your new detector using the freshly created outlier batch:

In [ ]:
od_preds = od.predict(X_outlier, return_instance_score=True)

In [ ]:
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {}'.format(f1))
cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

In [ ]:
plot_instance_score(od_preds, y_outlier, labels, od.threshold)

Saving your handiwork is simple. 

In [ ]:
save_detector(od, "outlier_detector")

Uploading the outlier detector to a Google Storage bucket.

Make sure to replace `<YOUR NAME>` with your name to prevent overwriting others artefacts.

In [ ]:
# Recursive copy this time as the OD is saved as a directory containing all the relevant binaries and parameters.
!gsutil cp -r outlier_detector gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/lr/

In [ ]:
OD_URI = f'gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/lr/outlier_detector/'
OD_NAME = 'mahalobanis-od'

od_config = {'deployment': DEPLOYMENT_NAME,
             'deployment_namespace': NAMESPACE,
             'namespace': 'seldon-logs',
             'params': {'drift_batch_size': None,
                        'env_secret_ref': None,
                        'event_source': f'io.seldon.serving.dev-seldondeployment-{DEPLOYMENT_NAME}-outlier',
                        'event_type': 'io.seldon.serving.inference.outlier',
                        'http_port': '8080',
                        'model_name': OD_NAME,
                        'protocol': 'seldon.http',
                        'reply_url': 'http://seldon-request-logger.seldon-logs',
                        'storage_uri': OD_URI,
                        'user_permission': None},
             'prom_scraping': None,
             'url': None}

In [ ]:
od_api = OutlierDetectorApi(auth())
od_api.create_outlier_detector_seldon_deployment(name=DEPLOYMENT_NAME,
                                                 namespace=NAMESPACE,
                                                 outlier_detector=od_config)